<a href="https://colab.research.google.com/github/nykekana/ASL_Mapping/blob/master/draft3_assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Functions

In [0]:
import cv2 
from matplotlib import pyplot as plt
import numpy as np
import os
import re

In [0]:
def top_three_target_images(q,distance_function):
  #used for all steps 1-4
  # returns a list of the filenames of the top three target images for a given query image q
  # parameters: 
  # q - query image id number 
  # disance_function: the disctance funciton used compares two images based on simple measures of their color (eg: color, shape, texture)
  # pic_files =[f for f in (os.listdir('./images')) if ".jpg" in f]
  
  # path = 'content/drive/My Drive/nyk2111_assignment2/images'
  pic_files =[f for f in (os.listdir('/content/drive/My Drive/nyk2111_assignment2/images')) if ".jpg" in f]
  
  
  # pic_files =[f for f in (os.listdir('/content/drive/My Drive/nyk2111_assignment2/images')) if ".jpg" in f]
  # print(pic_files)

  pic_files.sort() 
  all_top_three_targets=[]

  for i in range(len(pic_files)):
    # print(pic_files[i], pic_files[:i]+pic_files[i+1:])
    pic1 = pic_files[i]
    d_list=[]
    for pic2 in pic_files[:i]+pic_files[i+1:]:
      # print(pic1, pic2)
      d = distance_function(pic1,pic2) #different part
      d_list.append([pic1, pic2, d])
    
    d_list.sort(key = lambda x: x[2]) 
    all_top_three_targets.append(d_list[:3])
  return [row[1] for row in all_top_three_targets[q-1]]

# q=12

# print(top_three_target_images(q, combined_distance))

# re.findall(r'\d+', (str(top_three_target_images(q,combined_distance))))

In [0]:
def bottom_target_image(q, distance_function):
  #used for all steps 1-4
  # returns the filename of the target image most unlike q
  # parameters: 
  # q - query image id number 
  # disance_function: the disctance function used compares two images based on simple measures of their color (eg: color, shape, texture)
  # path = 'content/drive/My Drive/nyk2111_assignment2/images'
  pic_files =[f for f in (os.listdir('/content/drive/My Drive/nyk2111_assignment2/images')) if ".jpg" in f]
  pic_files.sort() 
  all_bottom_targets=[]

  for i in range(len(pic_files)):
    # print(pic_files[i], pic_files[:i]+pic_files[i+1:])
    pic1 = pic_files[i]
    d_list=[]
    for pic2 in pic_files[:i]+pic_files[i+1:]:
      d = distance_function(pic1,pic2)
      d_list.append([pic1, pic2, d])
    d_list.sort(key = lambda x: x[2]) 
    all_bottom_targets.append(d_list[-1])
  bottom_target = all_bottom_targets[q-1][1]
  return bottom_target
  
# print((bottom_target_image(3, combined_distance)))

In [0]:
def l1_distance(im_hist1, im_hist2, eps = 1e-10):
	# compute the l1 distance according to equation given on sheet
  # image functions were read as the histograms of each of the images 
  # parameters: 
  # 
  rows = 60
  cols = 89
  d = np.sum([np.abs((a - b)) / (2*rows*cols) for (a, b) in zip(im_hist1, im_hist2)])
  return d

In [0]:
def get_crowd_count_matrix():
  # coverts crowd.txt files into a parsable array 
  with open("/content/drive/My Drive/nyk2111_assignment2/Crowd.txt", "r") as file1:
      FileasList = file1.readlines()
      FileasList = ([line.split() for line in FileasList])
      return (FileasList)

# get_crowd_count_matrix()[0][2]
    

In [0]:
def get_borda_count(q,t):
  # returns the borda count of a specific query image q and target image t accroding to crowd.txt file
  return int(get_crowd_count_matrix()[q-1][t-1])
# get_borda_count(17,18)

In [0]:
def score(query_image, distance_func, crowd_count_type = get_borda_count):
  # 
  t1, t2, t3 = (re.findall(r'\d+', (str(top_three_target_images(query_image, distance_func)))))
  t1, t2, t3 = int(t1), int(t2), int(t3)
  # print(t1,t2, t3)
  return crowd_count_type(query_image, t1) + crowd_count_type(query_image, t2) + crowd_count_type(query_image, t3) 
  



In [0]:
def make_data_row(display_list, image_labels, row_score):
  q, t1,t2,t3,t40 = display_list
  # print(display_list)
  data_row = """ <tr>
    <td><p></p>""" + str(row_score) + """ </td>
    <td><img src=" """ + q + """ "><br>""" + image_labels[0] + """</td>
    <td><img src=" """ + t1 + """ "><br>""" + image_labels[1] + """</td>
    <td><img src=" """ + t2 + """ "><br>""" + image_labels[2] + """</td>
    <td><img src=" """ + t3 +""" "><br>""" + image_labels[3] + """</td>
    <td><img src=" """+ str(t40) +""" "><br>""" + str(image_labels[4]) + """</td>
    </tr>"""
  return data_row

In [0]:
def make_html_table(distance_function, score_crowd_count_type=get_borda_count):

  path = '/content/drive/My Drive/nyk2111_assignment2/images/'
  pic_files =[ path+f for f in (os.listdir('/content/drive/My Drive/nyk2111_assignment2/images')) if ".jpg" in f]

  pic_files.sort()

  data_rows = []
  # print(pic_files)
  for q_num,q in enumerate(pic_files,1):
    # print(q_num,q)
    t1,t2,t3 = (top_three_target_images(q_num, distance_function))
    t40 = bottom_target_image(q_num, distance_function)
    # print([q, t1,t2,t3, t40])
    row_score = score(q_num, distance_function, crowd_count_type=score_crowd_count_type)
    
    q_id = re.findall(r'\d+', (str(q)))[0]
    t1_id = re.findall(r'\d+', (str(t1)))[0]
    t2_id = re.findall(r'\d+', (str(t2)))[0]
    t3_id = re.findall(r'\d+', (str(t3)))[0]
    t40_id = re.findall(r'\d+', (str(t40)))[0]

    q_title = "query image: " + q_id
    t1_title= "target image: " + t1_id + "<br>" + "crowd count: "+ str(get_borda_count(int(q_id), int(t1_id)))
    t2_title= "target image: " + t2_id + "<br>" + "crowd count: "+ str(get_borda_count(int(q_id), int(t2_id)))
    t3_title= "target image: " + t3_id + "<br>" +  "crowd count: "+ str(get_borda_count(int(q_id), int(t3_id)))
    t40_title= "target image: " + t40_id + "<br>" + "crowd count: "+ str(get_borda_count(int(q_id), int(t40_id)))

    labels = [q_title, t1_title,t2_title,t3_title, t40_title]

    display_list = [q,t1,t2,t3,t40]
    row = make_data_row(display_list, labels, row_score)

    data_rows.append(row)
  all_data_rows = "".join(data_rows)
  return all_data_rows

  

In [0]:
def get_system_grand_score_sum(distance_func, score_crowd_count_type = get_borda_count):  
  all_scores_list = all_scores(distance_func) # (query image, score of query image)
  grand_system_score = [sum(i) for i in zip(*all_scores_list)] [1] # (sum(query images), sum (scores of query images) )
  return grand_system_score

In [0]:
def get_my_preferences_matrix():
  with open("/content/drive/My Drive/nyk2111_assignment2/MyPreferences.txt", "r") as file1:
      FileasList = file1.readlines()
      FileasList = ([line.split() for line in FileasList])
      return (FileasList)
# print(get_my_preferences_matrix())

def my_top_three_target_images(query_image):
  return get_my_preferences_matrix()[query_image-1][1:]

# print(my_top_three_target_images(1))

In [0]:
def matches(q, distance_function):
  system_top_three = re.findall(r'\d+', (str(top_three_target_images(q, distance_function))))
  my_top_three = my_top_three_target_images(q)

  return len(set(system_top_three) & set(my_top_three))

# print(matches(1))

In [0]:
def lapacian_image(im_name):
  ddepth = cv2.CV_16S #3 - 
  # [variables]
  # [load]
  imageName = im_name
  img = cv2.imread(imageName,0) # Load an image
  
  laplacian = cv2.Laplacian(img,cv2.CV_16U)

  return laplacian
# [display]

# plt.imshow(lapacian_image("i01.jpg"), cmap='gray')
# plt.show()



In [0]:
def binary_image(im_name):
  # Based off of step 3's instructions 
  # returns an image array of a binary image 
  intensity_image = lapacian_image(im_name) # intensity image is a grayscale image
  (thresh, blackAndWhiteImage) = cv2.threshold(intensity_image, 95, 255, cv2.THRESH_BINARY)
  return blackAndWhiteImage
# bw_im = binary_image("i01.jpg")
# plt.imshow(bw_im, cmap="gray")
# plt.axis('off')
# plt.show()

In [0]:
path = "/content/drive/My Drive/nyk2111_assignment2/images/"
def color_distance(image1_filename, image2_filename):
  #step 1

  # path = "/content/drive/My Drive/nyk2111_assignment2/images/"
  # image1_filename, image2_filename = path+image1_filename, path+image2_filename

  image1 = cv2.imread(image1_filename) 
  image2 = cv2.imread(image2_filename) 

  # print(image1, image2)

  image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
  image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)

  hist1 = cv2.calcHist([image1], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
  hist2 = cv2.calcHist([image2], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])

  d = l1_distance(hist1, hist2)
  return d 

def texture_distance(image1_filename, image2_filename):
  #step2 
  path = "/content/drive/My Drive/nyk2111_assignment2/images/"

  image1_filename, image2_filename = path+image1_filename, path+image2_filename

  image1 = lapacian_image(image1_filename)
  image2 = lapacian_image(image2_filename)

  hist1 = cv2.calcHist([image1],[0],None,[256],[0,256])
  hist2 = cv2.calcHist([image2],[0],None,[256],[0,256])

  d = l1_distance(hist1, hist2)
  return d
def shape_distance(image1_filename, image2_filename):
  #step 3
  path = "/content/drive/My Drive/nyk2111_assignment2/images/"

  image1_filename, image2_filename = path+image1_filename, path+image2_filename

  a, b = binary_image(image1_filename), binary_image(image2_filename)
  rows,cols = 60,89

  total_sum = []
  for x in range(rows):
    for y in range(cols):
        k = int(a[x,y] == b[x,y])
        total_sum.append(abs(k)/(60*89))
  return ( sum(total_sum) )

def combined_distance(image1_filename, image2_filename):
  # a,
  # path = "/content/drive/My Drive/nyk2111_assignment2/images/"

  # image1_filename, image2_filename = path+image1_filename, path+image2_filename

  q,t = re.findall(r'\d+', (image1_filename))[0], re.findall(r'\d+', (image2_filename))[0]
  q,t = int(q), int(t)

  color_d = color_distance(image1_filename, image2_filename)
  texture_d = texture_distance(image1_filename, image2_filename)
  shape_d = shape_distance(image1_filename, image2_filename)

  ground_truth = get_borda_count(q,t)/(6*65)

  all_d = [color_d, texture_d, shape_d]
  closest = (min(all_d, key=lambda x:abs(x-ground_truth)))

  if (closest == color_d):
    a,b,c = 0.5,0.25,0.25
  if (closest == texture_d):
    a,b,c = 0.25,0.5,0.25
  if (closest == shape_d):
    a,b,c = 0.25,0.25,0.5

  p = a*color_d + b*texture_d + c*shape_d
  return p

In [0]:
def all_scores(distance_func, score_crowd_count_type = get_borda_count):
  all_scores_list = []
  for i in range(1,41):
    all_scores_list.append((i, score(i, distance_func, crowd_count_type=score_crowd_count_type))) # (query image, score of query image)
  all_scores_list.sort(key = lambda x: x[1]) # in order according to score least to greatest
  return (all_scores_list)


# all_scores(distance_func=shape_distance)

In [0]:
def get_highest_score(distance_func, score_crowd_count_type = get_borda_count):
  all_scores_list = all_scores(distance_func, score_crowd_count_type = get_borda_count)
  highest_score = all_scores_list[0]
  return highest_score # (query image, score of query image)

def get_lowest_score(distance_func, score_crowd_count_type = get_borda_count):
  all_scores_list = all_scores(distance_func, score_crowd_count_type = get_borda_count)
  lowest_score = all_scores_list[-1]
  return lowest_score # (query image, score of query image)

In [0]:
def get_all_targets_match(distance_func):
  all_matches = get_all_matches(distance_function=distance_func)
  all_targets_match_query_list = []
  for i in all_matches: 
    num_of_matches = i[1]
    query_image = i[0]
    if num_of_matches == 3:
      all_targets_match_query_list.append(query_image)
  return all_targets_match_query_list

def get_no_targets_match(distance_func):
  all_matches = get_all_matches(distance_function=distance_func)
  no_targets_match_query_list = []
  for i in all_matches: 
    num_of_matches = i[1]
    query_image = i[0]
    if num_of_matches == 0:
      no_targets_match_query_list.append(query_image)
  return no_targets_match_query_list

In [0]:
#matches
def get_all_matches(distance_function):
  all_matches = []
  for i in range(1,41):
    all_matches.append((i, matches(i, distance_function))) # (query image, number of target matches )
  return(all_matches)

# print(get_all_matches(distance_function=color_distance))


# Results
---



# Step 1

In [129]:
!ls

shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
ls: cannot open directory '.': Transport endpoint is not connected



## System versus crowd preferences, using scores


In [130]:
data = make_html_table(distance_function=color_distance)
grand_system_score = get_system_grand_score_sum(distance_func=color_distance)
grand_system_score_html = "<h2>Scores Grand total: "+str(grand_system_score)+"</h2>"

results_table = """ <html><body><table>
    <tr>
        <th>Score</th>
        <th>Query Image</th>
        <th>target 1</th>
        <th>target 2</th>
        <th>target 3</th>
        <th>target 40</th>
      </tr> """ + data +""" <tr>"""+grand_system_score_html+""" </html></body></table>"""


with open("/content/drive/My Drive/nyk2111_assignment2/images/step1_results.html", 'w') as file: 
  file.write(results_table)

print("Done creating step 1 crowd-based accuracy table.....")

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-130-23b6f1820a38>", line 1, in <module>
    data = make_html_table(distance_function=color_distance)
  File "<ipython-input-113-dd647e10d690>", line 12, in make_html_table
    t1,t2,t3 = (top_three_target_images(q_num, distance_function))
  File "<ipython-input-106-0e8acf075250>", line 25, in top_three_target_images
    d = distance_function(pic1,pic2) #different part
  File "<ipython-input-119-25f6cfc3041d>", line 13, in color_distance
    image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
cv2.error: OpenCV(4.1.2) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packag

error: ignored

## System versus personal preferences, using set intersection

In [0]:
# step 1
distance_function = color_distance # step 1
grand_intersection_value1 = sum([matches(q, distance_function) for q in range(1,41) ])
print("For Step 1, grand intersection value is ", grand_intersection_value1)
# print('{0:.2f}%'.format((grand_intersection_value / 120 * 100)))


## Discussion

In [0]:
distance_function = color_distance # step 1

highest_color_score_query_image = get_highest_score(distance_function)[0]
lowest_color_score_query_image = get_lowest_score(distance_function)[0]

print("For Step 1, the system get the highest score for the following queries: ",highest_color_score_query_image)
print("For Step 1, the system get the lowest score for the following queries: ",lowest_color_score_query_image)

color_all_targets_match_query_list =  get_all_targets_match(distance_function)
color_no_targets_match_query_list = get_no_targets_match(distance_function)

print("For Step 1, all three of my personal preferences matched for the following queries: ",color_all_targets_match_query_list)
print("For Step 1, none of my three personal preferences matched for the following queries: ",color_no_targets_match_query_list)

# Step 2 


## System versus crowd preferences, using scores


In [0]:
data = make_html_table(distance_function=texture_distance)
texture_grand_system_score = get_system_grand_score_sum(distance_func=texture_distance)
grand_system_score_html = "<h2>Scores Grand total: "+str(texture_grand_system_score)+"</h2>"

results_table = """ <html><body><table>
    <tr>
        <th>Score</th>
        <th>Query Image</th>
        <th>target 1</th>
        <th>target 2</th>
        <th>target 3</th>
        <th>target 40</th>
      </tr> """ + data +""" <tr>"""+grand_system_score_html+""" </html></body></table>"""

# %cd /content/drive/My Drive/nyk2111_assignment2/images

with open("/content/drive/My Drive/nyk2111_assignment2/images/step2_results.html", 'w') as file: 
  file.write(results_table)

print("Done creating step 2 crowd-based accuracy table.....")

## System versus personal preferences, using set intersection

In [0]:
# step 2
distance_function = texture_distance # step 2

grand_intersection_value2 = sum([matches(q, distance_function) for q in range(1,41) ])
print("For Step 2, grand intersection value is ", grand_intersection_value2)
# print('{0:.2f}%'.format((grand_intersection_value / 120 * 100)))


## Discussion 

In [0]:
distance_function = texture_distance # step 2

highest_texture_score_query_image = get_highest_score(distance_function)[0]
lowest_texture_score_query_image = get_lowest_score(distance_function)[0]

print("For Step 2, the system get the highest score for the following queries: ",highest_texture_score_query_image)

print("For Step 2, the system get the lowest score for the following queries: ",lowest_texture_score_query_image)

color_all_targets_match_query_list =  get_all_targets_match(distance_function)
color_no_targets_match_query_list = get_no_targets_match(distance_function)

print("For Step 2, all three of my personal preferences matched for the following queries: ",color_all_targets_match_query_list)
print("For Step 2, none of my three personal preferences matched for the following queries: ",color_no_targets_match_query_list)

# Step 3


## System versus crowd preferences, using scores


In [0]:
data = make_html_table(distance_function=shape_distance)
shape_grand_system_score = get_system_grand_score_sum(distance_func=shape_distance)
grand_system_score_html = "<h2>Scores Grand total: "+str(shape_grand_system_score)+"</h2>"

results_table = """ <html><body><table>
    <tr>
        <th>Score</th>
        <th>Query Image</th>
        <th>target 1</th>
        <th>target 2</th>
        <th>target 3</th>
        <th>target 40</th>
      </tr> """ + data +""" <tr>"""+grand_system_score_html+""" </html></body></table>"""
# %cd /content/drive/My Drive/nyk2111_assignment2/images

with open("/content/drive/My Drive/nyk2111_assignment2/images/step3_results.html", 'w') as file: 
  file.write(results_table)

print("Done creating step 3 crowd-based accuracy table.....")

## System versus personal preferences, using set intersection

In [0]:
# step 3
distance_function = shape_distance # step 3

grand_intersection_value3 = sum([matches(q, distance_function) for q in range(1,41) ])
print("For Step 3, grand intersection value is ", grand_intersection_value3)
# print('{0:.2f}%'.format((grand_intersection_value / 120 * 100)))


## Discussion

In [0]:
distance_function = shape_distance # step 3

highest_distance_score_query_image = get_highest_score(distance_function)[0]
lowest_distance_score_query_image = get_lowest_score(distance_function)[0]

print("For Step 3, the system get the highest score for the following queries: ",highest_distance_score_query_image)
print("For Step 3, the system get the lowest score for the following queries: ",lowest_distance_score_query_image)

color_all_targets_match_query_list =  get_all_targets_match(distance_function)
color_no_targets_match_query_list = get_no_targets_match(distance_function)

print("For Step 3, all three of my personal preferences matched for the following queries: ",color_all_targets_match_query_list)
print("For Step 3, none of my three personal preferences matched for the following queries: ",color_no_targets_match_query_list)

# Step 4


## System versus crowd preferences, using scores


In [0]:
data = make_html_table(distance_function=combined_distance)
combined_grand_system_score = get_system_grand_score_sum(distance_func=combined_distance)
grand_system_score_html = "<h2>Scores Grand total: "+str(combined_grand_system_score)+"</h2>"

results_table = """ <html><body><table>
    <tr>
        <th>Score</th>
        <th>Query Image</th>
        <th>target 1</th>
        <th>target 2</th>
        <th>target 3</th>
        <th>target 40</th>
      </tr> """ + data +""" <tr>"""+grand_system_score_html+""" </html></body></table>"""

# %cd /content/drive/My Drive/nyk2111_assignment2/images

with open("/content/drive/My Drive/nyk2111_assignment2/images/step4_results.html", 'w') as file: 
  file.write(results_table)

print("Done creating step 4 crowd-based accuracy table.....")

## System versus personal preferences, using set intersection

In [0]:
# step 4
distance_function = combined_distance # step 4

grand_intersection_value4 = sum([matches(q, distance_function) for q in range(1,41) ])
print("For Step 4, grand intersection value is ", grand_intersection_value4)
# print('{0:.2f}%'.format((grand_intersection_value / 120 * 100)))


The values of a,b,c depend on how close each of the distances are to the ground truth. The closer the distance is, the more weight that distance has in the overall linear sum. 

## Discussion

In [0]:
distance_function = combined_distance # step 4

highest_color_score_query_image = get_highest_score(distance_function)[0]
lowest_color_score_query_image = get_lowest_score(distance_function)[0]

print("For Step 4, the system get the highest score for the following queries: ",highest_color_score_query_image)
print("For Step 4, the system get the lowest score for the following queries: ",lowest_color_score_query_image)

color_all_targets_match_query_list =  get_all_targets_match(distance_function)
color_no_targets_match_query_list = get_no_targets_match(distance_function)

print("For Step 4, all three of my personal preferences matched for the following queries: ",color_all_targets_match_query_list)
print("For Step 4, none of my three personal preferences matched for the following queries: ",color_no_targets_match_query_list)

# Step 5

## Crowd-based performance

### What is the actual upper bound on the very best grand total score that is possible with these images, given all the differences of opinion in Crowd.txt?

In [0]:
best_scores_all_distances_query_images = [get_system_grand_score_sum(color_distance),
                                          get_system_grand_score_sum(shape_distance),
                                          get_system_grand_score_sum(texture_distance), 
                                          get_system_grand_score_sum(combined_distance)]
upper_bound = max(best_scores_all_distances_query_images)

print("The actual upper bound (i.e.: the highest score of all the distances that this system can compute) is  ", upper_bound)


### How close (as a percentage) did your final system grand total score come to this actual upper bound?

In [0]:
system_diff = abs(combined_grand_system_score-upper_bound)/upper_bound*100
print("My final system, which I found from Step 4, got ", system_diff, "% close to this upper bound")


### How close (as a percentage) did your personal preference grand intersection value come to the maximum of 120?

In [0]:
personal_diff = abs(120-grand_intersection_value4)/120 *100
print("My personal preference grand intersection got ", personal_diff, "% close to this upper bound")

## User-based performance

### Replace Crowd.txt with a (very sparse) 40x40 matrix that was crowd-sourced based only on you. That is, each row has a single 3, a single 2, and a single 1, and sums to 6

In [0]:
def get_my_crowd(q,t):
  return int(make_my_crowd_matrix()[q-1][t-1])
def make_my_crowd_matrix():
  rows, cols = 40,40
  a = [[0 for x in range(1, 1+rows)] for y in range(1, 1+cols)]
  # print(b)

  for q in range(1, 1+rows):
    t1,t2,t3 = my_top_three_target_images(q)
    t1,t2,t3 = int(t1), int(t2), int(t3)

    row_num = q-1
    # print("Row: ", row_num)
    # print([t1,t2,t3])
    a[q-1][t1-1] = 1
    a[q-1][t2-1] = 2
    a[q-1][t3-1] = 3

    if (sum(a[row_num]) is not 6):
      print("OH NO!", sum(a[row_num]))
  return a

def write_my_crowd():
  a = make_my_crowd_matrix()
  mat = np.matrix(a)
  with open('/content/drive/My Drive/nyk2111_assignment2/MyCrowd.txt','wb') as f:
      for line in mat:
          np.savetxt(f, line, fmt='%.2f')
q=11
t=7
print(my_top_three_target_images(q))
print(make_my_crowd_matrix()[q-1][t-1])

In [0]:
write_my_crowd()

### Redo Step 4–but keeping your decisions in Steps 1, 2, and 3, the same–by simply searching for the values of (a, b, c) that make you happiest

In [0]:
print(score(1,combined_distance)) # out of 354
print(score(1,combined_distance, crowd_count_type=get_my_crowd)) # out of 9

print(score(1,combined_distance)/354)
print(score(1,combined_distance, crowd_count_type=get_my_crowd)/9) # out of 9

#### System versus crowd preferences, using scores

In [0]:
data = make_html_table(distance_function=combined_distance, score_crowd_count_type=get_my_crowd)
combined_grand_system_score = get_system_grand_score_sum(distance_func=combined_distance, score_crowd_count_type= get_my_crowd)
grand_system_score_html = "<h2>Scores Grand total: "+str(combined_grand_system_score)+"</h2>"


results_table = """ <html><body><table>
    <tr>
        <th>Score</th>
        <th>Query Image</th>
        <th>target 1</th>
        <th>target 2</th>
        <th>target 3</th>
        <th>target 40</th>
      </tr> """ + data +""" <tr>"""+grand_system_score_html+""" </html></body></table>"""



# %cd /content/drive/My Drive/nyk2111_assignment2/images

with open("step5_results.html", 'w') as file: 
  file.write(results_table)

print("Done creating step 5 crowd-based accuracy table.....")

#### System versus personal preferences, using set intersection 

In [0]:
# step 5
distance_function = combined_distance # step 5

grand_intersection_value5 = sum([matches(q, distance_function) for q in range(1,41) ])
print("For Step 5, grand intersection value is ", grand_intersection_value5)
# print('{0:.2f}%'.format((grand_intersection_value / 120 * 100)))


#### Discussion

In [0]:
distance_function = combined_distance # step 5

highest_score_query_image = get_highest_score(distance_function, score_crowd_count_type = get_my_crowd)[0]
lowest_score_query_image = get_lowest_score(distance_function)[0]

print("For Step 5, the system get the highest score for the following queries: ",highest_color_score_query_image)
print("For Step 5, the system get the lowest score for the following queries: ",lowest_color_score_query_image)

color_all_targets_match_query_list =  get_all_targets_match(distance_function)
color_no_targets_match_query_list = get_no_targets_match(distance_function)

print("For Step 5, all three of my personal preferences matched for the following queries: ",color_all_targets_match_query_list)
print("For Step 5, none of my three personal preferences matched for the following queries: ",color_no_targets_match_query_list)

### How much closer (as a percentage) did your personal preference grand intersection value now come to the maximum of 120? Discuss why it was not higher.

In [0]:
personal_diff = abs(120-grand_intersection_value5)/120 *100
print("My personal preference grand intersection got ", personal_diff, "% close to this upper bound")